#**How to Train YOLOv12 Object Detection Model on a Custom Dataset**

In [ ]:
!nvidia-smi

**Step 01 # Install the Ultralytics Package**

In [ ]:
!pip install ultralytics

**Step 02 # Import All the Requried Libraries**

In [ ]:
import os
import ultralytics
ultralytics.checks()

In [ ]:
from ultralytics import YOLO
from IPython.display import Image

In [ ]:
HOME = os.getcwd()
print(HOME)

In [ ]:
!pwd

In [ ]:
# rm -rf

**Step # 03 Download Dataset from Roboflow**

https://universe.roboflow.com/project-uyrxf/ppe_detection-v1x3l/dataset/2

In [ ]:
!pip install roboflow   # Requires restart after installation.

In [ ]:
from roboflow import Roboflow
rf = Roboflow(api_key="z1uD0b9XOZdnivUdFkEd")
project = rf.workspace("project-uyrxf").project("ppe_detection-v1x3l")
version = project.version(2)
dataset = version.download("yolov12")

In [ ]:
!ls -la {dataset.location}

**Step # 04 Fine-tune YOLOv12 model on a Custom Dataset**

**NOTE:** We need to make a few changes to our downloaded dataset so it will work with YOLOv12. Run the following bash commands to prepare your dataset for training by updating the relative paths in the `data.yaml` file, ensuring it correctly points to the subdirectories for your dataset's `train`, `test`, and `valid` subsets.

In [ ]:
!sed -i '$d' {dataset.location}/data.yaml
!sed -i '$d' {dataset.location}/data.yaml
!sed -i '$d' {dataset.location}/data.yaml
!sed -i '$d' {dataset.location}/data.yaml
!echo -e "test: ../test/images\ntrain: ../train/images\nval: ../valid/images" >> {dataset.location}/data.yaml

In [ ]:
!cat {dataset.location}/data.yaml

We are now ready to fine-tune our YOLOv12 model. In the code below, we initialize the model using a starting checkpoint—here, we use `yolov12s.yaml`, but you can replace it with any other model (e.g., `yolov12n.pt`, `yolov12m.pt`, `yolov12l.pt`, or `yolov12x.pt`) based on your preference. We set the training to run for 50 epochs in this example; however, you should adjust the number of epochs along with other hyperparameters such as batch size, image size, and augmentation settings (scale, mosaic, mixup, and copy-paste) based on your hardware capabilities and dataset size.

**Note:** **Note that after training, you might encounter a `TypeError: argument of type 'PosixPath' is not iterable error` — this is a known issue, but your model weights will still be saved, so you can safely proceed to running inference.**

In [ ]:
# NOTE: This uses YOLO vlln for some unknown reason.
model = YOLO("yolo12s.yaml")
# model = YOLO("yolo12s.pt")

In [ ]:
# %cd /

In [ ]:
# ! cat root/.config/Ultralytics/settings.json

In [ ]:
results = model.train(data=f'{dataset.location}/data.yaml', epochs=50)

**Step # 05 Evaluate fine-tuned YOLOv12 model**


In [ ]:
import locale
locale.getpreferredencoding = lambda: "UTF-8"

!ls {HOME}/runs/detect/train3/

In [ ]:
Image(filename=f'{HOME}/runs/detect/train3/confusion_matrix.png', width=1000)

In [ ]:
Image(filename=f'{HOME}/runs/detect/train3/confusion_matrix_normalized.png', width=1000)

In [ ]:
Image(filename=f'{HOME}/runs/detect/train3/results.png', width=1000)

Precision = TP / (TP + FP)

Precision is simply true positives out of total detections.


In [ ]:
Image(filename=f'{HOME}/runs/detect/train3/P_curve.png', width=1000)

Recall = TP / (TP + FN)

Recall is the True Positive out of all Ground Truths

In [ ]:
Image(filename=f'{HOME}/runs/detect/train3/R_curve.png', width=1000)

In [ ]:
Image(filename=f'{HOME}/runs/detect/train3/train_batch0.jpg', width=1000)

In [ ]:
Image(filename=f'{HOME}/runs/detect/train3/val_batch0_pred.jpg', width=1000)

In [ ]:
Image(filename=f'{HOME}/runs/detect/train3/val_batch1_pred.jpg', width=1000)

**Step # 06 Download the Model Weights from the Google Drive**

In [ ]:
!gdown "https://drive.google.com/uc?id=1ka7Gj8RE6iP8-ExtigZJhmiyr5k5lqRQ&confirm=t"

**Step # 07  Validate Fine-Tuned Model**

In [ ]:
model = YOLO("best.pt")  # load a custom model

# Validate the model
metrics = model.val()  # no arguments needed, dataset and settings remembered
metrics.box.map  # map50-95
metrics.box.map50  # map50
metrics.box.map75  # map75
metrics.box.maps  # a list contains map50-95 of each category

**Step # 08 Inference with Custom Model on Images**

In [ ]:
dataset.location

In [ ]:
results = model.predict(source = f"{dataset.location}/test/images", save = True)

In [ ]:
import glob
import os
from IPython.display import Image as IPyImage, display

latest_folder = max(glob.glob(f'{HOME}/runs/detect/predict*/'), key=os.path.getmtime)

for img in glob.glob(f'{latest_folder}/*.jpg')[2:5]:
    display(IPyImage(filename=img, width=600))
    print("\n")

**Step # 09 Inference with Custom Model on Videos**

In [ ]:
import locale
locale.getpreferredencoding = lambda: "UTF-8"

In [ ]:
!gdown "https://drive.google.com/uc?id=1Nyl18zr7zOJHh8uHtSic3dsJSY4LI_t-&confirm=t"

In [ ]:
!gdown "https://drive.google.com/uc?id=1SxcrDanPAr-PZfJjWBxoa2t8lUSM2dZn&confirm=t"

In [ ]:
results = model.predict(source = f"/{HOME}/PPE_Part1.mp4", save = True)

In [ ]:
!rm '/content/result_compressed.mp4'

In [ ]:
from IPython.display import HTML
from base64 import b64encode
import os

# Input video path
save_path = f'{HOME}/runs/detect/predict/PPE_Part1.avi'

# Compressed video path
compressed_path = "/content/result_compressed.mp4"

os.system(f"ffmpeg -i {save_path} -vcodec libx264 {compressed_path}")

# Show video
mp4 = open(compressed_path,'rb').read()
data_url = "data:video/mp4;base64," + b64encode(mp4).decode()

HTML("""
<video width=400 controls>
      <source src="%s" type="video/mp4">
</video>
""" % data_url)

In [ ]:
results = model.predict(source = f"/{HOME}/PPE_Part2.mp4", save = True)

In [ ]:
!rm '/content/result_compressed.mp4'

In [ ]:
from IPython.display import HTML
from base64 import b64encode
import os

# Input video path
save_path = f'{HOME}/runs/detect/predict/PPE_Part2.avi'

# Compressed video path
compressed_path = "/content/result_compressed.mp4"

os.system(f"ffmpeg -i {save_path} -vcodec libx264 {compressed_path}")

# Show video
mp4 = open(compressed_path,'rb').read()
data_url = "data:video/mp4;base64," + b64encode(mp4).decode()
HTML("""
<video width=400 controls>
      <source src="%s" type="video/mp4">
</video>
""" % data_url)